# OSMnx

https://osmnx.readthedocs.io/en/stable/

OSMnx is a Python package that lets you download geospatial data from OpenStreetMap and model, project, visualize, and analyze real-world street networks and any other geospatial geometries.


In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [ ]:
ox.settings.use_cache = True
ox.settings.log_console=True

### Retrieve Street Network from OSM Data

* drive - get drivable public streets (but not service roads)
* drive_service - get drivable streets, including service roads
* walk - get all streets and paths that pedestrians can use (this network type ignores one-way directionality)
* bike - get all streets and paths that cyclists can use
* all - download all non-private OSM streets and paths
* all_private - download all OSM streets and paths, including private-access ones


Functions:


* graph.graph_from_bbox
* graph.graph_from_address
* graph.graph_from_place
* graph.graph_from_point



In [ ]:
place = "Prague, Czech Republic"
graph = ox.graph.graph_from_place(place, network_type='drive')

In [ ]:
fig, ax = ox.plot.plot_graph(graph)

#### Save Graph as GeoPackage

There are two layers:

* edges
* nodes


In [ ]:
ox.io.save_graph_geopackage(graph, filepath='geodata/network-shape.gpkg')

In [ ]:
nodes, streets = ox.graph_to_gdfs(graph)

In [ ]:
len(streets)

In [ ]:
streets.head()

In [ ]:
street_types = pd.DataFrame(streets["highway"].apply(pd.Series)[0].value_counts().reset_index())
street_types.columns = ["type", "count"]

In [ ]:
street_types

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
sns.barplot(y="type", x="count", data=street_types, ax=ax)
plt.tight_layout()
plt.show();

### Putting the Street Network on a Folium Map

In [ ]:
m = folium.Map([50.06233320433081, 14.428682853905626],
zoom_start=13,tiles="CartoDB dark_matter")

jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}
folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)
m

### Routing

There are two train stations in Basel, lets find the route between them:

In [ ]:
start_location = ox.geocode('New Town, 110 00 Prague 1, Czechia')
start_location

In [ ]:
end_location = ox.geocode('Nusle, 140 00 Prague 4, Czechia')
end_location

#### Let's add them as POI to the map **above**:

In [ ]:
m = folium.Map([50.06233320433081, 14.428682853905626], zoom_start=13,tiles="CartoDB dark_matter")

#jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}
folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)

folium.Marker(start_location, 
              popup="Start",
              icon=folium.Icon(color="green", prefix="fa", icon="star")).add_to(m)

folium.Marker(end_location, 
              popup="End",
              icon=folium.Icon(color="red", prefix="fa", icon="star")).add_to(m)
m

In [ ]:
origin_node = ox.distance.nearest_nodes(graph, start_location[1], start_location[0])  # G, X, Y    Longitude first!
destination_node = ox.distance.nearest_nodes(graph, end_location[1], end_location[0])

#### Find the route between the nodes

Using NetworkX module to calculate

In [ ]:
route = nx.shortest_path(graph, origin_node, destination_node)

In [ ]:
fig, ax = ox.plot_graph_route(graph, route)

In [ ]:
from shapely.geometry import LineString

line = []
for i in route:
    point = (graph.nodes[i]['x'],graph.nodes[i]['y'])
    line.append(point)
    print(point)
    
ls = LineString(line)

#### Create a GeoDataFrame from the LineString (optional)


In [ ]:
gdf_route = pd.DataFrame(data= [['route 1',ls]],columns=['route','geometry'])
gdf_route

In [ ]:
gdf_route = gpd.GeoDataFrame(gdf_route,geometry='geometry')
gdf_route

In [ ]:
gdf_route.plot();

Convert this route to GeoJSON

In [ ]:
jsonroute = gdf_route.to_json()

In [ ]:
m = folium.Map([50.06233320433081, 14.428682853905626], zoom_start=13,tiles="CartoDB dark_matter")

#jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}

route_style = {'color': '#FF0000',
              'weight' : '4'}

folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)
folium.GeoJson(jsonroute, style_function=lambda x: route_style).add_to(m)

folium.Marker(start_location, 
              popup="Start",
              icon=folium.Icon(color="green", prefix="fa", icon="star")).add_to(m)

folium.Marker(end_location, 
              popup="End",
              icon=folium.Icon(color="red", prefix="fa", icon="star")).add_to(m)
m